In [1]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn import datasets
import cupy as cp

In [2]:
def my_cdist_func(A, B):
    LA, WA = A.shape
    LB, WB = B.shape
    print("A.shape",A.shape)
    print("B.shape",B.shape)
        
    ret_array = np.zeros((LA,LB))
    for i in range(LA):
        aux = np.array([A[i]-B[j] for j in range(LB)])
        ret_array[i] = np.sqrt(np.sum(np.power(aux,2),axis=1))
    
    print("aux.shape",aux.shape)
    
    return ret_array

In [13]:
### Possivel Versão CuPy 
# Não testei isso, não tenho CUDA aqui

def cp_cdist(A, B, metric='euclidean'):
    LA, WA = A.shape
    LB, WB = B.shape
    ret_array = cp.zeros((LA,LB))
    
    if metric == 'euclidean':
        aux = cp.empty((LB,WB))
        
        for i in range(LA):
            for j in range(LB):
                aux[j] = A[i]-B[j]
            ret_array[i] = cp.sqrt(cp.sum(cp.power(aux,2),axis=1))
    
    if metric == 'cosine':
        num = cp.empty((LB))
        den = cp.empty((LB))
        
        for i in range(LA):
            for j in range(LB):
                num [j]= cp.sum(A[i]*B[j])
                den [j]= cp.sqrt(cp.sum(A[i]*A[i])*cp.sum(B[j]*B[j]))
            ret_array[i] = 1 - num/den
    
    return ret_array

In [14]:
def my_cdist_cossine(A, B, metric='euclidean'):
    LA, WA = A.shape
    LB, WB = B.shape
    
    print("A.shape",A.shape)
    print("B.shape",B.shape)
    
    ret_array = np.zeros((LA,LB))
    for i in range(LA):
        num = np.array([np.sum(A[i]*B[j]) for j in range(LB)])
        den = np.array([np.sqrt(np.sum(A[i]*A[i])*np.sum(B[j]*B[j])) for j in range(LB)])
        ret_array[i] = 1 - num/den
    
    return ret_array

In [15]:
# Aqui eu coloquei uma matriz qlqr pra testar, ve se vc tem algo ai
iris = datasets.load_iris()
data = iris.data
L, W = data.shape

A = data[:int(L/3)]
B = data[int(L/3):]

C = cp.asarray(A)
D = cp.asarray(B)


In [23]:
my_cdist = my_cdist_func(A,B)

ValueError: non-scalar numpy.ndarray cannot be used for fill

In [17]:
my_cdist.shape

(50, 100)

In [18]:
my_cossine = my_cdist_cossine(A,B)

A.shape (50, 4)
B.shape (100, 4)


In [19]:
my_cossine.shape

(50, 100)

In [24]:
euclidean = cp.asnumpy(cp_cdist(C,D))

In [25]:
euclidean.shape

(50, 100)

In [20]:
cp_cdist_result = cp.asnumpy(cp_cdist(C,D,'cosine'))

In [21]:
cp_cdist_result.shape

(50, 100)

In [22]:
compare = (cp_cdist_result == my_cossine)

cont = 0

for i in range(len(compare)):
    for j in range(len(compare[0])):
        if compare[i,j] == False:
            print('difference:',my_cossine[i,j]-cp_cdist_result[i,j])
            print('position: (',i,',',j,')')
            print('_._'*20)
            
            cont += 1

print ('percent of difference: ', (cont*100/(my_cossine.shape[0]*my_cossine.shape[1])))

difference: -1.1102230246251565e-16
position: ( 0 , 0 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 0 , 2 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 0 , 4 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 0 , 5 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 0 , 9 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 0 , 11 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 0 , 12 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 0 , 15 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 0 , 16 )
_.__.__._

position: ( 6 , 30 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 3.3306690738754696e-16
position: ( 6 , 31 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 6 , 34 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 4.440892098500626e-16
position: ( 6 , 35 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 6 , 36 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 2.220446049250313e-16
position: ( 6 , 37 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 6 , 38 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 6 , 39 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 6 , 42 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.

position: ( 10 , 33 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 10 , 35 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 10 , 36 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 10 , 37 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 2.220446049250313e-16
position: ( 10 , 38 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 10 , 39 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 10 , 40 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 10 , 41 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 10 , 44 )
_.__.__.__.__.__.__.__.__.__.__.__

position: ( 16 , 70 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 16 , 74 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 16 , 76 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 16 , 77 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 16 , 78 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 16 , 79 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 16 , 82 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 16 , 83 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 16 , 84 )
_.__.__.__.__.__.__.__.__.__._

_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 22 , 93 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 22 , 94 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 22 , 98 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 22 , 99 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 23 , 0 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 23 , 4 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 23 , 9 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 23 , 10 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__

position: ( 28 , 35 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 28 , 36 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 28 , 37 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 28 , 38 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 28 , 40 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 28 , 41 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 28 , 42 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 28 , 44 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 28 , 45 )
_.__.__.__.__.__.__.__.__.__.__

position: ( 34 , 9 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 34 , 10 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 2.220446049250313e-16
position: ( 34 , 11 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -3.3306690738754696e-16
position: ( 34 , 13 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 2.220446049250313e-16
position: ( 34 , 14 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -3.3306690738754696e-16
position: ( 34 , 17 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 34 , 22 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 34 , 23 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 34 , 24 )
_.__.__.__.__.__.__.__.__.__.__.__

_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 38 , 73 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 38 , 75 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 38 , 76 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 38 , 77 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 38 , 78 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 38 , 81 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 38 , 83 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 38 , 84 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.

position: ( 45 , 26 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 45 , 28 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 1.1102230246251565e-16
position: ( 45 , 29 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 2.220446049250313e-16
position: ( 45 , 30 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 3.3306690738754696e-16
position: ( 45 , 31 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: 2.220446049250313e-16
position: ( 45 , 38 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -3.3306690738754696e-16
position: ( 45 , 42 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -2.220446049250313e-16
position: ( 45 , 44 )
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
difference: -1.1102230246251565e-16
position: ( 45 , 46 )
_.__.__.__.__.__.__.__.__.__.__._

array([[ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [False, False,  True, ...,  True,  True,  True]])